# **MOVIE RECOMMENDER**

Approach->Content Based

**PART 1: Data Preprocessing**

In [ ]:
#Importing Libraries
import numpy as np
import pandas as pd
import ast
import nltk

In [ ]:
#Importing the 2 datasets
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [ ]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
#Merging the 2 datasets into 1 dataset for CONVENIENCE
dataset=movies.merge(credits,on='title')
dataset.shape

(4809, 23)

In [ ]:
#ELIMINATING UNNECESSARY COLUMNS from our dataset
#These are the 7 columns (out of 23) which will be required: genre, id, keywords, title, overview, cast, crew;
#The other parameters (columns) were rejected as they are not required in our project's workflow as such
#The eliminated columns consisted mostly of numeric data which is not much required for a RECOMMENDER SYSTEM
dataset=dataset[['movie_id','title','overview','genres','keywords','cast','crew']]
dataset.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
#Removing Rows which have MISSING VALUES
dataset.dropna(inplace=True)
#Checking the count of missing values in all columns
dataset.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [ ]:
#DUPLICATE Check
dataset.duplicated().sum()

0

In [ ]:
#Converting 'overview' column values into a list
dataset['overview']=dataset['overview'].apply(lambda x:x.split())

In [ ]:
#Converting 'genres' and 'keywords' by extracting only the fields associated with each film
def convert(object):
  L=[]
  for i in ast.literal_eval(object):
    L.append(i['name'])
  return L

dataset['genres']=dataset['genres'].apply(convert)
dataset['keywords']=dataset['keywords'].apply(convert)

In [ ]:
#Same with 'cast' we will only keep list of the first 6 actors associated with it
def convert2(object):
  L=[]
  count=0;
  for i in ast.literal_eval(object):
    if count!=6:
      L.append(i['name'])
      count=count+1
    else:
      break
  return L

dataset['cast']=dataset['cast'].apply(convert2)

In [ ]:
#For the 'crew' column we only need the director
def convert3(object):
  L=[]
  for i in ast.literal_eval(object):
    if i['job']=='Director':
      L.append(i['name'])
      break;
  return L

dataset['crew']=dataset['crew'].apply(convert3)

In [ ]:
#Removing all spaces
dataset['genres']=dataset['genres'].apply(lambda x:[i.replace(" ","") for i in x] )
dataset['keywords']=dataset['keywords'].apply(lambda x:[i.replace(" ","") for i in x] )
dataset['cast']=dataset['cast'].apply(lambda x:[i.replace(" ","") for i in x] )
dataset['crew']=dataset['crew'].apply(lambda x:[i.replace(" ","") for i in x] )

In [ ]:
#MERGING the columns; overview+genres+keywords+cast+crew=tags
dataset['tags']=dataset['overview']+dataset['genres']+dataset['keywords']+dataset['cast']+dataset['crew']
dataset=dataset[['movie_id','title','tags']]

In [ ]:
dataset['tags']=dataset['tags'].apply(lambda x:" ".join(x))
dataset['tags']=dataset['tags'].apply(lambda x: x.lower())

<ipython-input-78-217ed9d331a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['tags']=dataset['tags'].apply(lambda x:" ".join(x))
<ipython-input-78-217ed9d331a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['tags']=dataset['tags'].apply(lambda x: x.lower())


In [ ]:
dataset.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


**PART 2: Model Building->NATURAL LANGUAGE PROCESSING**

In [ ]:
#Now for our RECOMMENDATION SYSTEM, we use method of NLP->Text Vectorization
#This is somewhat similar to a Bag-of-Words Model

In [ ]:
#STEMMING->Process of removing suffixes from words and converting them to their root/stem word,i.e.[loves,loving,loved]-->[love,love,love]
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

#Creating a function for Stemming a text message
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
#Applying stem to our dataset
dataset['tags']=dataset['tags'].apply(stem)

In [ ]:
#Importing our function for text vectorization from python package 'Scikit Learn'
#We also remove the STOPWORDS->english words which have significantly zero contribution to the statement
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(stop_words='english',max_features=10000)
vectors=cv.fit_transform(dataset['tags']).toarray()

#Note that 'vector' will be SPARSE MATRIX

In [ ]:
#Features
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zookeep', 'zoëkravitz',
       'zuleikharobinson'], dtype=object)

In [ ]:
#Each movie represents a vector in the multidimensional space
#For recommending movies similar to the original movie, we use the nearest vectors
#CURSE OF DIMENSIONALITY->For multidimensions, euclidean distance often fails hence we use the acute angle between 2 vectors to compare their similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)
similarity

array([[1.        , 0.06693161, 0.07045671, ..., 0.03888079, 0.        ,
        0.        ],
       [0.06693161, 1.        , 0.05134962, ..., 0.02125256, 0.        ,
        0.02231054],
       [0.07045671, 0.05134962, 1.        , ..., 0.02237187, 0.        ,
        0.        ],
       ...,
       [0.03888079, 0.02125256, 0.02237187, ..., 1.        , 0.03774257,
        0.03888079],
       [0.        , 0.        , 0.        , ..., 0.03774257, 1.        ,
        0.07924289],
       [0.        , 0.02231054, 0.        , ..., 0.03888079, 0.07924289,
        1.        ]])

In [ ]:
#Now we come to the main part,i.e. RECOMMENDING 12 movies similar to a given movie
def recommend(movie):
    movie_index=dataset[dataset['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:13]

    for i in movies_list:
        print(dataset.iloc[i[0]].title)

In [ ]:
#Prints the most relevant 12 films for our example film
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Titan A.E.
Independence Day
Battle: Los Angeles
Predators
Meet Dave
U.F.O.
Jupiter Ascending
Small Soldiers
Lifeforce


In [ ]:
recommend('Batman Begins')

The Dark Knight
Batman
The Dark Knight Rises
Batman
Rockaway
Batman v Superman: Dawn of Justice
Defendor
Synecdoche, New York
10th & Wolf
Sexy Beast
Amidst the Devil's Wings
American Psycho


**PART 3: WEB APP DEPLOYMENT**

In [ ]:
import pickle

In [ ]:
pickle.dump(dataset,open('Movies.pkl','wb'))

In [ ]:
pickle.dump(dataset.to_dict(),open('movies_list.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

# **THE END**